In [ ]:
import pandas as pd
from tinyolap.database import Database

from tinyolap.cell import Cell
from tinyolap.decorators import rule
from tinyolap.rules import RuleScope
from tinyolap.view import View
import os

In [ ]:
db = Database("forms", in_memory=True) 
db.caching = False

In [ ]:
# названия файлов с измерениями
dimension_names = ["dim_company", "dim_assets", "dim_date"]
cwd = os.getcwd()

dimensions = []

for name in dimension_names:
    # строим путь до файла
    file_name = os.path.join(cwd, "data", f"{name}.csv")
    
    # создаем новое измерение в TinyOLAP и открываем его на редактирование
    dim = db.add_dimension(name).edit()
    df = pd.read_csv(file_name)
    
    # делаем список уникальных элементов измерения
    unique_dim = df[df.columns[0]].unique().astype(str)
    
    # добавляем в наше созданное измерение
    dim.add_many(unique_dim)
    
    # добавляем аггрегирующий элемент, который включает все остальные
    if len(unique_dim) > 1:
        dim.add_many("All", tuple(unique_dim))

    # "закрепляем" измения измерения
    dim.commit()

In [ ]:
data_file_name = os.path.join(cwd, "data", "data.csv")

df = pd.read_csv(data_file_name)

measures_target_names = ["price", "quantity", "amount"]

# создаем общий показатель "measures"
measures = db.add_dimension("measures").edit()

# добавляем в него элементы измерений из исходных данных 
measures.add_many(measures_target_names)

# "закрепляем" измения измерения
measures.commit() 
dimension_names.append('measures')

In [ ]:
# создаем OLAP куб. В качестве измерений передадим все созданные измерения и показатели
cube = db.add_cube(name="data", dimensions=dimension_names)
    
dim_count = 3

for i in range(dim_count):
    df[df.columns[i]] = df[df.columns[i]].astype(str)

# заполняем данные из файлы. Путь для каждого значения иммеет вид
# (измерение1,...,измерениеn,показатель1)
# (измерение1,...,измерениеn,показательn)
for row in df.itertuples(index=False):
    for cur_measure_index, cur_measure in enumerate(measures_target_names):
        address = tuple(row[:dim_count]) + (cur_measure, )
        value = row.__getattribute__(cur_measure)
        cube.set(address, value)

In [ ]:
view = View(cube, definition={
        "filters": {"dimensions": ["dim_assets", "dim_date"]},
        "columns": {"dimensions": ["measures"]},
        "rows": {"dimensions": ["dim_company"]}
    })

print(view.to_console_output())

In [ ]:
view = View(cube, definition={
        "filters": {"dimensions": [{"dimension": "dim_assets", "members": "All"}, {"dimension": "dim_date", "members": "All"}]},
        "columns": {"dimensions": ["measures"]},
        "rows": {"dimensions": ["dim_company"]}
    })

print(view.to_console_output())

In [ ]:
@rule("data", trigger=["measures:amount"], scope=RuleScope.BASE_LEVEL)
def rule_cost(c: Cell):
    quantity = c["quantity"]
    price = c["price", c.BYPASS_RULES]
    if quantity and price:
        return float(price) * float(quantity)
    return c.CONTINUE

@rule("data", trigger=["measures:price"], scope=RuleScope.AGGREGATION_LEVEL)
def rule_price(c: Cell):
    quantity = c["quantity"]
    amount = c["amount"]
    if quantity != 0:
        return float(amount / quantity)
    return c.CONTINUE

cube.register_rule(rule_cost)
cube.register_rule(rule_price)

In [ ]:
view = View(cube, definition={
        "filters": {"dimensions": [{"dimension": "dim_assets", "members": "All"}, {"dimension": "dim_date", "members": "All"}]},
        "columns": {"dimensions": ["measures"]},
        "rows": {"dimensions": ["dim_company"]}
    })

print(view.to_console_output())

In [ ]:
print(view.to_dict())